<div align=center>
		
<font size=5>
In the Name of God
<font/>
<p></p>
<br/>
<br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mahdieh Soleimani
</font>
<br/>
<br/>
Fall 2022

<hr/>
<font color=red>
CSP Assignment (20 points)
</font>
</div>

In this assignment you will solve a CSP.

In [1]:
# do not import other packages for this assignment
import numpy as np

In this problem, you're gonna be in role of a party host and your task is to manage the dinner table setup. You have to fill the dishes and determine the guests' seats. You can see the table schema in the following picture. 

<img src="CSP.jpg" width="800"/> <br/>
  - There will be $n$ dishes and $2n$ guests. <br/>
  - Each dish will be filled with a specific kind of food. <br/>

<h1> Constraints </h1>
- Each dish will be filled with a specific kind of food. <br/>
- Each guest likes certain foods and won't eat other foods. <br/>
- The guests won't be able to move the dishes but they can pass food to each other (Seat j can pass food to seat j-1 and seat j+1. e.g: seat 3 can pass food to seat 2 and seat 4 so if seat 5 wants the food in dish 3, first, the food should be passed to seat 4 and then to seat 5). <br/>
<h2>The big problem is ... </h2>
- Some of the guests have conflicts with others (direction is important). If they sit next to these guests, they won't pass food to them (In previous example, if person in seat 4 had conflict with person in seat 5, person in seat 4 would't pass food to him). <br/>

<h1> Goal </h1>
Yout goal is to determine the guests' seats in a way that no one leaves your party, hungry.
 <h1>Note</h1>
  - You must implement and use AC-3 algorithm. <br/>
  - You will probably need to utilize the heuristic algorithms you have learned (e.g. MRV and LCV) or consider nearly tree-structured graphs approach in order to pass all tests in an appropriate time.

## Input
  - The first line consists of $N$ (count of dishes)
  - In the following $2N$ lines, the $i$ th line ($1 \le i \le 2N$) is a list of preferred foods for the $i$ th guest (separated by space).
  - In the next line, $M$, the total number of exit constraints (conflicts between 2 guest) are given.
  - in each of the following $M$ lines, a pair of numbers (separated by space) is given (direction is important e.g: "1 4" means that guest 1 has conflict with guest 4 but the reverse is not necessarily true).
$$1 \le N \le 50$$

## Output
In the only line, print:
  - list of one appropriate assignment ordered by guest number (separated by space).
  - `NO` if there is no solution.

In [36]:
global n, favourite_foods, conflicts
n = 6
variables = [{x: True for x in range(2 * n)} for i in range(2 * n)]  # set of variable for csp problem
unassigned = {x: True for x in range(2 * n)}  # set of unassigned variable
favourite_foods = [[5],
                   [4],
                   [3],
                   [2],
                   [1],
                   [0],
                   [5],
                   [4],
                   [3],
                   [2],
                   [1],
                   [0]]
conflicts = [[11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]]



def does_new_guest_block(step, variables):
    if step < n:
        for i in range(step + 1):
            seated = (list(variables[i].keys()))[0]
            left_conflict = -1
            for j in range(i):
                near_0 = (list(variables[j].keys()))[0]
                near_1 = (list(variables[j + 1].keys()))[0]
                if near_1 in conflicts[near_0]:
                    left_conflict = j

            illegal_foods = 0
            if left_conflict != -1:
                for food in favourite_foods[seated]:
                    if food <= left_conflict or food > step:
                        illegal_foods = illegal_foods + 1
                if illegal_foods == len(favourite_foods[i]):
                    return True

    else:
        for i in range(step + 1):
            seated = (list(variables[i].keys()))[0]
            if i < n:
                left_conflict = n
                for j in range(i, n - 1):
                    near_0 = (list(variables[j].keys()))[0]
                    near_1 = (list(variables[j + 1].keys()))[0]
                    if near_0 in conflicts[near_1]:
                        left_conflict = j + 1
                        break
                    
                illegal_foods = 0
                for food in favourite_foods[seated]:
                    if food >= left_conflict:
                        illegal_foods = illegal_foods + 1
                if illegal_foods == len(favourite_foods[seated]):
                    return True
            else:
                left_conflict = -1
                for j in range(n, i - 1):
                    near_0 = (list(variables[j].keys()))[0]
                    near_1 = (list(variables[j + 1].keys()))[0]
                    if near_1 in conflicts[near_0]:
                        left_conflict = j - n
                    if left_conflict != 0:
                        for food in favourite_foods[seated]:
                            if food > seated or food <= left_conflict:
                                illegal_foods = illegal_foods + 1
                            if illegal_foods == len(favourite_foods[seated]):
                                return True

    return False
                    


def can_guest_be_placed(unassigned, right_upper_conflict, right_lower_conflict):
    for food in favourite_foods[unassigned]:
        if food > right_upper_conflict or food > right_lower_conflict:
            return True
    return False


def right_conflicts(step, variables):
    right_upper_conflict = -1
    right_lower_conflict = -1
    if step < n:
        return -1, -1

    for i in range(step):
        variable = (list(variables[i].keys()))[0]
        #print(variable, end=" ")
    #print()

    for i in range(n - 1):
        variable = (list(variables[i].keys()))[0]
        right_variable = (list(variables[i + 1].keys()))[0]
        if right_variable in conflicts[variable]:
            right_upper_conflict = i
    
    for i in range(n, step - 1):
        variable = (list(variables[i].keys()))[0]
        right_variable = (list(variables[i + 1].keys()))[0]
        if right_variable in conflicts[variable]:
            right_lower_conflict = i - n
        
    #print(right_upper_conflict, right_lower_conflict)
    return right_upper_conflict, right_lower_conflict


def remove_incosistent_values(unassigned_varibale_values, unassigned_variable_index, assigned_variable_index,
                              assigned_variable_value, right_upper_conflict, right_lower_conflict, step, pruned_edges):
    pruned = []
    if assigned_variable_value in unassigned_varibale_values:
        unassigned_varibale_values.pop(assigned_variable_value)
        pruned.append([unassigned_variable_index, assigned_variable_value])

    illegal_unassigned = []
    if step >= n:  # check if unassaigned guest cant be placed # ATTENTION: if a guest cant be placed here, it cant be plced anywhere
        for unassigned in unassigned_varibale_values:
            if can_guest_be_placed(unassigned, right_upper_conflict, right_lower_conflict) is False:
                illegal_unassigned.append(unassigned)

    for unassigned in illegal_unassigned:
        unassigned_varibale_values.pop(unassigned)
        pruned.append([unassigned_variable_index, unassigned])

    illegal_unassigned = []
    if step >= n and (unassigned_variable_index == 2 * n - 1 or unassigned_variable_index == step + 1):
        for unassigned in unassigned_varibale_values:
            poops = right_upper_conflict
            coops = right_lower_conflict
            if unassigned_variable_index == 2 * n - 1:
                variable = (list(variables[n - 1].keys()))[0]
                if unassigned in conflicts[variable]:
                    poops = n - 1
            if unassigned_variable_index == step + 1:
                variable = (list(variables[step].keys()))[0]
                if unassigned in conflicts[variable]:
                    coops = step - n
            if can_guest_be_placed(unassigned, poops, coops) is False:
                    illegal_unassigned.append(unassigned)

    for unassigned in illegal_unassigned:
        unassigned_varibale_values.pop(unassigned)
        pruned.append([unassigned_variable_index, unassigned])
    
    illegal_unassigned = []
    if step < n and pruned_edges:
        if unassigned_variable_index == step + 1:
            for unassigned in unassigned_varibale_values:
                variable_0 = (list(variables[step].keys()))[0]
                if variable_0 in conflicts[unassigned]:
                    illegal_unassigned.append(unassigned)
    
    if step >= n and pruned_edges:
        if unassigned_variable_index == 2 * n - 1:
            for unassigned in unassigned_varibale_values:
                variable_0 = (list(variables[step].keys()))[0]
                if variable_0 in conflicts[unassigned]:
                    illegal_unassigned.append(unassigned)

    
    for unassigned in illegal_unassigned:
        unassigned_varibale_values.pop(unassigned)
        pruned.append([unassigned_variable_index, unassigned])
    return pruned


def forward_checking(variables, assigned_index, assigned_value, unassigned, right_upper_conflict, right_lower_conflict, step, pruned_edges):
    pruned = []
    for index in unassigned:
        pruned += remove_incosistent_values(variables[index], index, assigned_index, assigned_value, right_upper_conflict, right_lower_conflict, step, pruned_edges)
    return pruned


def Party(variables, step, unassigned):
    if step == 2 * n:
        return True, variables

    variable, minimum = 0, 2 * n + 1
    for i in unassigned:
        if len(variables[i]) == 0:
            return False, variables

        if len(variables[i]) < minimum:
            minimum = len(variables[i])
            variable = i
    variable = step
    unassigned.pop(variable)
    temp = {x: True for x in variables[variable]}

    sorted_values = (list(variables[variable].keys()))

    for value in sorted_values:
        variables[variable] = {value: True}
        right_upper_conflict, right_lower_conflict = right_conflicts(step, variables)
        pruned_edges = does_new_guest_block(step, variables)
        pruned = forward_checking(variables, variable, value, unassigned, right_upper_conflict, right_lower_conflict, step, pruned_edges)
        find, soln = Party(variables, step + 1, unassigned)
        if find:
            return True, soln
        for pair in pruned:
            variables[pair[0]][pair[1]] = True  # restore pruned varibales
    variables[variable] = temp  # restore chessboard
    unassigned[variable] = True
    return False, variables



n = int(input())
conflicts = [[] for i in range(2 * n)]
variables = [{x: True for x in range(2 * n)} for i in range(2 * n)]  # set of variable for csp problem
unassigned = {x: True for x in range(2 * n)}  # set of unassigned variable
favourite_foods = []
for i in range(2 * n):
    q = input().split()
    fave= []
    for food in q:
        fave.append(int(food) - 1)
    favourite_foods.append(fave)

m = int(input())
for i in range(m):
    q = input().split()
    q_0 = int(q[0]) - 1
    q_1 = int(q[1]) - 1
    conflicts[q_0].append(q_1)

find, soln = Party(variables, 0, unassigned)

if find is False:
    print("No")
else:
    for i in range(2 * n):
        ans = (list(soln[i].keys()))[0] + 1
        print(ans, end=" ")



2 3 4 1 

An extreme example of the problem is solved here (every guest has conflict with all other geusts and overyone has only one favourite food), so the problem has the most constraints.
The only possible answer is that every guest seats in front of their favourite food.
I have assumed that the table is circular (guest $n$ can pass food to geust $2n$.)
Favourite food of $i$ th guest is $n - (i - 1).mod.(n)$.
(Foods and guest are zero-indexed below.)

In [34]:
n = 10
variables = [{x: True for x in range(2 * n)} for i in range(2 * n)]  # set of variable for csp problem
unassigned = {x: True for x in range(2 * n)}  # set of unassigned variable
favourite_foods = [[9],
                   [8],
                   [7],
                   [6],
                   [5],
                   [4],
                   [3],
                   [2],
                   [1],
                   [0],
                   [9],
                   [8],
                   [7],
                   [6],
                   [5],
                   [4],
                   [3],
                   [2],
                   [1],
                   [0]]
conflicts = [[19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
             [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]]

find, soln = Party(variables, 0, unassigned)

if find is False:
    print("No")
else:
    for i in range(2 * n):
        ans = (list(soln[i].keys()))[0] + 1
        print(ans, end=" ")

10 19 18 7 6 5 4 3 2 11 20 9 8 17 16 15 14 13 12 1 

As we see above, every guest is directly in front of their favourite food. (It takse around 16s to run for n = 10.)